In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 3.4 MB/s eta 0:00:00


In [2]:
import gradio as gr
import os
import json
import uuid
from datetime import datetime
from groq import Groq

In [3]:
import os

os.environ["GROQ_API_KEY"] = "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"
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
client = Groq(api_key=GROQ_API_KEY)

In [4]:
# Default system prompt
SYSTEM_PROMPT = (
    "You are an intelligent, friendly, and highly adaptable Teaching Assistant Chatbot. "
    "Your mission is to help users of all ages and skill levels—from complete beginners to seasoned professionals—learn Python, Data Science, and Artificial Intelligence. "
    "You explain concepts clearly using real-world analogies, examples, and interactive exercises. "
    "You ask questions to assess the learner's level, adapt accordingly, and provide learning paths tailored to their pace and goals. "
    "Your responses are structured, engaging, and supportive. "
    "You can explain code snippets, generate exercises and quizzes, and recommend projects. "
    "You never overwhelm users with jargon. Instead, you scaffold complex concepts in simple, digestible steps."
)

In [5]:
LEARNING_PATHS = {
    "python_beginner": {
        "title": "Python Fundamentals",
        "description": "Learn Python basics from variables to functions",
        "modules": [
            "Variables & Data Types",
            "Control Flow",
            "Functions",
            "Data Structures",
            "File I/O"
        ]
    },
    "python_intermediate": {
        "title": "Intermediate Python",
        "description": "Advance your Python skills with OOP and more",
        "modules": [
            "Object-Oriented Programming",
            "Modules & Packages",
            "Error Handling",
            "List Comprehensions",
            "Decorators & Generators"
        ]
    },
    "data_science_beginner": {
        "title": "Data Science Foundations",
        "description": "Begin your data science journey",
        "modules": [
            "Numpy Basics",
            "Pandas Fundamentals",
            "Data Visualization",
            "Basic Statistics",
            "Intro to Machine Learning"
        ]
    },
    "data_science_advanced": {
        "title": "Advanced Data Science",
        "description": "Master complex data science concepts",
        "modules": [
            "Advanced ML Algorithms",
            "Feature Engineering",
            "Time Series Analysis",
            "Natural Language Processing",
            "Deep Learning Basics"
        ]
    },
    "ai_specialization": {
        "title": "AI Specialization",
        "description": "Focus on artificial intelligence concepts",
        "modules": [
            "Neural Networks",
            "Computer Vision",
            "Advanced NLP",
            "Reinforcement Learning",
            "AI Ethics"
        ]
    },
    "generative_ai": {
        "title": "Generative AI",
        "description": "Learn how to build and work with generative AI systems",
        "modules": [
            "Generative Models Overview",
            "GANs & Diffusion Models",
            "Large Language Models",
            "Prompt Engineering",
            "Fine-tuning & RLHF"
        ]
    },
    "agentic_ai": {
        "title": "Agentic AI Systems",
        "description": "Explore AI systems that can act autonomously",
        "modules": [
            "Foundations of AI Agents",
            "Planning & Decision Making",
            "Tool-using AI Systems",
            "Multi-agent Architectures",
            "Human-AI Collaboration"
        ]
    }
}

In [6]:
LEARNING_RESOURCES = {
    "Visual": {
        "python_beginner": [
            {"title": "Python Crash Course Visual Guide", "url": "https://nostarch.com/pythoncrashcourse2e"},
            {"title": "Video Course: Python for Everybody", "url": "https://www.py4e.com/"},
            {"title": "Python Visualizations and Infographics", "url": "https://python-graph-gallery.com/"},
            {"title": "Visual Studio Code Python Tutorial", "url": "https://code.visualstudio.com/docs/python/python-tutorial"}
        ],
        "python_intermediate": [
            {"title": "Fluent Python with Visual Examples", "url": "https://www.oreilly.com/library/view/fluent-python-2nd/9781492056348/"},
            {"title": "Python Design Patterns Visualized", "url": "https://refactoring.guru/design-patterns/python"},
            {"title": "Interactive Python Visualizer", "url": "https://pythontutor.com/"},
            {"title": "Visual Guide to Python Testing", "url": "https://pragprog.com/titles/bopytest/python-testing-with-pytest/"}
        ],
        "data_science_beginner": [
            {"title": "Data Visualization with Python and Seaborn", "url": "https://seaborn.pydata.org/tutorial.html"},
            {"title": "Kaggle Learn: Data Visualization", "url": "https://www.kaggle.com/learn/data-visualization"},
            {"title": "DataCamp Python Data Visualization", "url": "https://www.datacamp.com/courses/introduction-to-data-visualization-with-python"},
            {"title": "Plotly Python Graphing Library", "url": "https://plotly.com/python/"}
        ],
        "data_science_advanced": [
            {"title": "Visualization in Machine Learning", "url": "https://machinelearningmastery.com/data-visualization-for-machine-learning/"},
            {"title": "Visual Hands-On Machine Learning", "url": "https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/"},
            {"title": "Stanford ML: Visual Guide to Neural Networks", "url": "https://see.stanford.edu/Course/CS229"},
            {"title": "Animated ML Algorithm Visualizations", "url": "https://www.youtube.com/c/3blue1brown"}
        ],
        "ai_specialization": [
            {"title": "DeepLearning.AI Video Courses", "url": "https://www.deeplearning.ai/"},
            {"title": "TensorFlow Playground", "url": "https://playground.tensorflow.org/"},
            {"title": "Visual Guide to Neural Networks", "url": "https://pytorch.org/tutorials/"},
            {"title": "GANs Explained Visually", "url": "https://poloclub.github.io/ganlab/"}
        ],
        "generative_ai": [
            {"title": "Visualizing Large Language Models", "url": "https://karpathy.ai/zero-to-hero.html"},
            {"title": "Diffusion Models Visual Guide", "url": "https://huggingface.co/learn/diffusion-models/"},
            {"title": "Visual Prompt Engineering Guide", "url": "https://www.promptingguide.ai/"},
            {"title": "Stable Diffusion Visual Tutorial", "url": "https://stability.ai/learn"}
        ],
        "agentic_ai": [
            {"title": "Visual Guide to LangChain", "url": "https://python.langchain.com/docs/get_started/introduction"},
            {"title": "Illustrated AutoGen Guide", "url": "https://microsoft.github.io/autogen/"},
            {"title": "Visual Multi-Agent Simulations", "url": "https://www.anthropic.com/research/debate"},
            {"title": "Animated Reinforcement Learning", "url": "https://rail.eecs.berkeley.edu/deeprlcourse/"}
        ]
    },
    "Reading/Writing": {
        "python_beginner": [
            {"title": "Python Documentation", "url": "https://docs.python.org/3/"},
            {"title": "Real Python Text Tutorials", "url": "https://realpython.com/"},
            {"title": "Automate the Boring Stuff with Python", "url": "https://automatetheboringstuff.com/"},
            {"title": "Think Python (Free eBook)", "url": "https://greenteapress.com/wp/think-python-2e/"}
        ],
        "python_intermediate": [
            {"title": "Fluent Python (Book)", "url": "https://www.oreilly.com/library/view/fluent-python-2nd/9781492056348/"},
            {"title": "Effective Python (Book)", "url": "https://effectivepython.com/"},
            {"title": "Python Cookbook (Book)", "url": "https://www.oreilly.com/library/view/python-cookbook-3rd/9781449357337/"},
            {"title": "Full Stack Python (Text Tutorials)", "url": "https://www.fullstackpython.com/"}
        ],
        "data_science_beginner": [
            {"title": "Python Data Science Handbook", "url": "https://jakevdp.github.io/PythonDataScienceHandbook/"},
            {"title": "Towards Data Science (Articles)", "url": "https://towardsdatascience.com/"},
            {"title": "Introduction to Statistical Learning", "url": "https://www.statlearning.com/"},
            {"title": "Data Science from Scratch (Book)", "url": "https://www.oreilly.com/library/view/data-science-from/9781492041122/"}
        ],
        "data_science_advanced": [
            {"title": "Machine Learning Mastery (Text Tutorials)", "url": "https://machinelearningmastery.com/"},
            {"title": "Deep Learning Book", "url": "https://www.deeplearningbook.org/"},
            {"title": "Elements of Statistical Learning", "url": "https://web.stanford.edu/~hastie/ElemStatLearn/"},
            {"title": "Dive into Deep Learning", "url": "https://d2l.ai/"}
        ],
        "ai_specialization": [
            {"title": "Artificial Intelligence: A Modern Approach", "url": "http://aima.cs.berkeley.edu/"},
            {"title": "Deep Learning (Book)", "url": "https://www.deeplearningbook.org/"},
            {"title": "Stanford ML Course Notes", "url": "https://see.stanford.edu/Course/CS229"},
            {"title": "ArXiv Machine Learning Papers", "url": "https://arxiv.org/list/cs.LG/recent"}
        ],
        "generative_ai": [
            {"title": "LLM Introduction Paper", "url": "https://arxiv.org/abs/2303.18223"},
            {"title": "Generative AI Guide (eBook)", "url": "https://www.oreilly.com/library/view/generative-deep-learning/9781492041931/"},
            {"title": "Prompt Engineering Guide", "url": "https://www.promptingguide.ai/"},
            {"title": "Stanford CS324: LLM Course Notes", "url": "https://stanford-cs324.github.io/winter2022/"}
        ],
        "agentic_ai": [
            {"title": "LangChain Documentation", "url": "https://python.langchain.com/docs/get_started/introduction"},
            {"title": "Agentic AI Papers Collection", "url": "https://arxiv.org/abs/2304.03442"},
            {"title": "Multi-Agent Debate Research", "url": "https://www.anthropic.com/research/debate"},
            {"title": "Reinforcement Learning: An Introduction", "url": "http://incompleteideas.net/book/the-book-2nd.html"}
        ]
    },
    "Hands-on Projects": {
        "python_beginner": [
            {"title": "Project-Based Python Tutorial", "url": "https://projectbasedpython.com/"},
            {"title": "Exercism: Python Track", "url": "https://exercism.org/tracks/python"},
            {"title": "Python Project Ideas with Code", "url": "https://github.com/topics/python-projects"},
            {"title": "Build 5 Mini Python Projects", "url": "https://www.freecodecamp.org/news/python-projects-for-beginners/"}
        ],
        "python_intermediate": [
            {"title": "Django Project Tutorial", "url": "https://docs.djangoproject.com/en/stable/intro/tutorial01/"},
            {"title": "Flask Mega-Tutorial", "url": "https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world"},
            {"title": "Python Project Cookbook", "url": "https://pythonprojectcookbook.com/"},
            {"title": "Real-world Python Projects", "url": "https://realpython.com/tutorials/projects/"}
        ],
        "data_science_beginner": [
            {"title": "Kaggle: Intro to Machine Learning", "url": "https://www.kaggle.com/learn/intro-to-machine-learning"},
            {"title": "Data Science Projects with Python", "url": "https://github.com/PacktPublishing/Data-Science-Projects-with-Python"},
            {"title": "DataCamp Projects", "url": "https://www.datacamp.com/projects"},
            {"title": "Practical Data Analysis Projects", "url": "https://www.dataquest.io/data-science-projects/"}
        ],
        "data_science_advanced": [
            {"title": "Applied Machine Learning Projects", "url": "https://github.com/practical-tutorials/project-based-learning#python"},
            {"title": "Kaggle Competitions", "url": "https://www.kaggle.com/competitions"},
            {"title": "Building ML Pipelines", "url": "https://www.oreilly.com/library/view/building-machine-learning/9781492053187/"},
            {"title": "ML Project Walkthroughs", "url": "https://machinelearningmastery.com/start-here/#projects"}
        ],
        "ai_specialization": [
            {"title": "TensorFlow Tutorials & Projects", "url": "https://www.tensorflow.org/tutorials"},
            {"title": "PyTorch Projects Collection", "url": "https://pytorch.org/tutorials/beginner/pytorch_with_examples.html"},
            {"title": "Hugging Face Project Walkthroughs", "url": "https://huggingface.co/learn"},
            {"title": "Computer Vision Projects", "url": "https://www.pyimagesearch.com/"}
        ],
        "generative_ai": [
            {"title": "Build Your Own LLM Application", "url": "https://buildyourowngpt.com/"},
            {"title": "Generative Art Projects", "url": "https://genart.tech/"},
            {"title": "LangChain Project Tutorials", "url": "https://python.langchain.com/docs/get_started/introduction"},
            {"title": "Fine-tuning LLMs: Hands-on Guide", "url": "https://huggingface.co/blog/how-to-train"}
        ],
        "agentic_ai": [
            {"title": "Build an AI Agent with LangChain", "url": "https://python.langchain.com/docs/use_cases/autonomous_agents"},
            {"title": "AutoGen Projects", "url": "https://microsoft.github.io/autogen/docs/examples/"},
            {"title": "Building Autonomous AI Systems", "url": "https://github.com/yoheinakajima/babyagi"},
            {"title": "Tool-using AI Projects", "url": "https://github.com/hwchase17/langchain-experiments"}
        ]
    },
    "Video Tutorials": {
        "python_beginner": [
            {"title": "CS50's Introduction to Programming with Python", "url": "https://cs50.harvard.edu/python/"},
            {"title": "freeCodeCamp Python Course", "url": "https://www.freecodecamp.org/learn/scientific-computing-with-python/"}
        ],
        "python_intermediate": [
            {"title": "MIT OpenCourseWare: Python", "url": "https://ocw.mit.edu/courses/6-0001-introduction-to-computer-science-and-programming-in-python-fall-2016/"}
        ],
        "data_science_beginner": [
            {"title": "freeCodeCamp Data Analysis Course", "url": "https://www.freecodecamp.org/learn/data-analysis-with-python/"}
        ],
        "data_science_advanced": [
            {"title": "Machine Learning Course by Andrew Ng", "url": "https://www.coursera.org/learn/machine-learning"},
            {"title": "Deep Learning Specialization", "url": "https://www.deeplearning.ai/deep-learning-specialization/"}
        ],
        "ai_specialization": [
            {"title": "MIT 6.S191: Introduction to Deep Learning", "url": "http://introtodeeplearning.com/"}
        ],
        "generative_ai": [
            {"title": "Neural Networks: Zero to Hero", "url": "https://karpathy.ai/zero-to-hero.html"},
            {"title": "Prompt Engineering for LLMs", "url": "https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/"}
        ],
        "agentic_ai": [
            {"title": "Building AI Agents with LangChain", "url": "https://www.youtube.com/watch?v=iw2Wcw7qPuE"},
            {"title": "AutoGPT and Multi-Agent Systems", "url": "https://www.youtube.com/watch?v=4YaILFaUXTo"}
        ]
    },
    "Interactive Exercises": {
        "python_beginner": [
            {"title": "CodeCademy Python Course", "url": "https://www.codecademy.com/learn/learn-python-3"},
            {"title": "CheckiO Python Challenges", "url": "https://py.checkio.org/"},
            {"title": "Python Tutor", "url": "https://pythontutor.com/"},
            {"title": "HackerRank Python Practice", "url": "https://www.hackerrank.com/domains/python"}
        ],
        "python_intermediate": [
            {"title": "Exercism Python Track", "url": "https://exercism.org/tracks/python"},
            {"title": "CodeWars Python Challenges", "url": "https://www.codewars.com/?language=python"},
            {"title": "LeetCode Python Problems", "url": "https://leetcode.com/problemset/all/?difficulty=EASY&page=1&languageTags=python"},
            {"title": "Project Euler", "url": "https://projecteuler.net/"}
        ],
        "data_science_beginner": [
            {"title": "DataCamp Interactive Courses", "url": "https://www.datacamp.com/courses/free-introduction-to-r"},
            {"title": "Kaggle Learn Interactive Tutorials", "url": "https://www.kaggle.com/learn/overview"},
            {"title": "DataQuest Interactive Data Science", "url": "https://www.dataquest.io/"},
            {"title": "Google's Data Analytics Course", "url": "https://www.coursera.org/professional-certificates/google-data-analytics"}
        ],
        "data_science_advanced": [
            {"title": "Interactive ML Course", "url": "https://www.coursera.org/learn/machine-learning"},
            {"title": "Kaggle Interactive Competitions", "url": "https://www.kaggle.com/competitions"},
            {"title": "Interactive Deep Learning", "url": "https://www.deeplearning.ai/courses/"},
            {"title": "Machine Learning Playground", "url": "https://ml-playground.com/"}
        ],
        "ai_specialization": [
            {"title": "TensorFlow Playground", "url": "https://playground.tensorflow.org/"},
            {"title": "Interactive Neural Network Builder", "url": "https://alexlenail.me/NN-SVG/"},
            {"title": "AI Experiments with Google", "url": "https://experiments.withgoogle.com/collection/ai"},
            {"title": "OpenAI Gym", "url": "https://www.gymlibrary.dev/"}
        ],
        "generative_ai": [
            {"title": "Hugging Face Spaces", "url": "https://huggingface.co/spaces"},
            {"title": "Interactive LLM Playground", "url": "https://platform.openai.com/playground"},
            {"title": "Interactive Stable Diffusion", "url": "https://huggingface.co/spaces/stabilityai/stable-diffusion"},
            {"title": "GPT-4 Interactive Demos", "url": "https://chat.openai.com/"}
        ],
        "agentic_ai": [
            {"title": "LangChain Interactive Tutorials", "url": "https://python.langchain.com/docs/get_started/introduction"},
            {"title": "Interactive AI Agent Builder", "url": "https://github.com/microsoft/TaskMatrix"},
            {"title": "AutoGen Playground", "url": "https://microsoft.github.io/autogen/"},
            {"title": "Reinforcement Learning Interactive Course", "url": "https://www.coursera.org/specializations/reinforcement-learning"}
        ]
    },
    "Combination": {
        "python_beginner": [
            {"title": "Python Documentation", "url": "https://docs.python.org/3/"},
            {"title": "Real Python", "url": "https://realpython.com/"},
            {"title": "Python for Everybody", "url": "https://www.py4e.com/"},
            {"title": "Automate the Boring Stuff with Python", "url": "https://automatetheboringstuff.com/"}
        ],
        "python_intermediate": [
            {"title": "Fluent Python", "url": "https://www.oreilly.com/library/view/fluent-python-2nd/9781492056348/"},
            {"title": "Python Design Patterns", "url": "https://refactoring.guru/design-patterns/python"},
            {"title": "Full Stack Python", "url": "https://www.fullstackpython.com/"},
            {"title": "Python Testing with pytest", "url": "https://pragprog.com/titles/bopytest/python-testing-with-pytest/"}
        ],
        "data_science_beginner": [
            {"title": "Kaggle Learn", "url": "https://www.kaggle.com/learn"},
            {"title": "Towards Data Science", "url": "https://towardsdatascience.com/"},
            {"title": "DataCamp", "url": "https://www.datacamp.com/"},
            {"title": "Python Data Science Handbook", "url": "https://jakevdp.github.io/PythonDataScienceHandbook/"}
        ],
        "data_science_advanced": [
            {"title": "Machine Learning Mastery", "url": "https://machinelearningmastery.com/"},
            {"title": "Hands-On Machine Learning with Scikit-Learn", "url": "https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/"},
            {"title": "Fast.ai", "url": "https://www.fast.ai/"},
            {"title": "Stanford CS229: Machine Learning", "url": "https://see.stanford.edu/Course/CS229"}
        ],
        "ai_specialization": [
            {"title": "DeepLearning.AI", "url": "https://www.deeplearning.ai/"},
            {"title": "TensorFlow Tutorials", "url": "https://www.tensorflow.org/tutorials"},
            {"title": "PyTorch Tutorials", "url": "https://pytorch.org/tutorials/"},
            {"title": "Hugging Face Course", "url": "https://huggingface.co/learn"}
        ],
        "generative_ai": [
            {"title": "Andrej Karpathy's Neural Networks Course", "url": "https://karpathy.ai/zero-to-hero.html"},
            {"title": "Hugging Face Diffusion Models Course", "url": "https://huggingface.co/learn/diffusion-models/"},
            {"title": "Prompt Engineering Guide", "url": "https://www.promptingguide.ai/"},
            {"title": "Stanford CS324: Large Language Models", "url": "https://stanford-cs324.github.io/winter2022/"}
        ],
        "agentic_ai": [
            {"title": "LangChain Documentation", "url": "https://python.langchain.com/docs/get_started/introduction"},
            {"title": "Microsoft AutoGen", "url": "https://microsoft.github.io/autogen/"},
            {"title": "Multi-Agent Debate by Anthropic", "url": "https://www.anthropic.com/research/debate"},
            {"title": "Berkeley CS285: Deep Reinforcement Learning", "url": "https://rail.eecs.berkeley.edu/deeprlcourse/"}
        ]
    }
}

In [7]:
PROJECT_IDEAS = {
    "Visual": {
        "python_beginner": [
            "Data Visualization Dashboard with Matplotlib",
            "Interactive Game with Pygame",
            "Visual Timer Application with Tkinter",
            "Color Palette Generator",
            "Image Processing Tool"
        ],
        "python_intermediate": [
            "Data Visualization Web App with Flask and D3.js",
            "Interactive Map Application",
            "Animated Data Dashboard",
            "Custom Visualization Library",
            "Image Recognition System"
        ],
        "data_science_beginner": [
            "Interactive Data Dashboard with Plotly",
            "Visual Exploratory Data Analysis Tool",
            "Chart Comparison Application",
            "Geographic Data Visualization",
            "Statistical Visualization Gallery"
        ],
        "data_science_advanced": [
            "Real-time Visual Analytics Dashboard",
            "Machine Learning Model Visualizer",
            "Neural Network Visualization Tool",
            "Computer Vision Project",
            "Interactive Data Storytelling Platform"
        ],
        "ai_specialization": [
            "Neural Network Architecture Visualizer",
            "Interactive AI Learning Environment",
            "Computer Vision Object Detector",
            "Visual Pattern Recognition System",
            "Brain-Computer Interface Visualization"
        ],
        "generative_ai": [
            "Style Transfer Art Generator",
            "Visual AI Art Gallery",
            "Image Generation Dashboard",
            "Interactive Text-to-Image System",
            "Visual Prompt Engineering Tool"
        ],
        "agentic_ai": [
            "Visual Agent Environment Simulator",
            "Agent Decision Tree Visualizer",
            "Multi-Agent Interaction Visualization",
            "Visual Reinforcement Learning Playground",
            "Interactive Agent Behavior Explorer"
        ]
    },
    "Reading/Writing": {
        "python_beginner": [
            "Text File Processing Tool",
            "Personal Journal Application",
            "Notes Organization System",
            "Simple Blog Platform",
            "Document Analyzer"
        ],
        "python_intermediate": [
            "Advanced Text Editor",
            "Markdown Documentation Generator",
            "Content Management System",
            "Personal Wiki Platform",
            "Technical Documentation Tool"
        ],
        "data_science_beginner": [
            "Text Data Analysis Tool",
            "Literature Review Database",
            "Research Paper Summarizer",
            "Study Notes Organizer",
            "Data Analysis Report Generator"
        ],
        "data_science_advanced": [
            "Research Paper Recommendation System",
            "Advanced NLP Analysis Tool",
            "Automated Report Generator",
            "Literature Review AI Assistant",
            "Technical Writing Assistant"
        ],
        "ai_specialization": [
            "Text Summarization System",
            "AI-Powered Document Analysis",
            "Scientific Paper Classification Tool",
            "Sentiment Analysis for Literature",
            "Technical Writing Enhancement System"
        ],
        "generative_ai": [
            "AI Writing Assistant",
            "Creative Story Generator",
            "Academic Paper Generator",
            "LLM-Powered Documentation Tool",
            "Custom Prompt Engineering Workbook"
        ],
        "agentic_ai": [
            "AI Research Assistant Agent",
            "Technical Documentation Generator",
            "Writing Style Analyzer",
            "Text-Based Agent Environment",
            "AI-Powered Knowledge Management System"
        ]
    },
    "Hands-on Projects": {
        "python_beginner": [
            "Weather App with API Integration",
            "To-Do List Application",
            "Simple Calculator",
            "Basic Web Scraper",
            "File Organizer Tool"
        ],
        "python_intermediate": [
            "REST API with Flask or Django",
            "Web Application with User Authentication",
            "Automated Testing Framework",
            "Command-Line Tool with Click",
            "Desktop Application with PyQt"
        ],
        "data_science_beginner": [
            "Exploratory Data Analysis Project",
            "Basic Machine Learning Model",
            "Data Cleaning Pipeline",
            "Simple Predictive Model",
            "Dataset Visualization Tool"
        ],
        "data_science_advanced": [
            "End-to-End Machine Learning Pipeline",
            "Model Deployment with Flask/FastAPI",
            "Time Series Forecasting Application",
            "Natural Language Processing Tool",
            "Recommendation System"
        ],
        "ai_specialization": [
            "Custom Neural Network Implementation",
            "Image Classification Application",
            "NLP Chatbot",
            "Reinforcement Learning Environment",
            "Computer Vision Project"
        ],
        "generative_ai": [
            "Fine-tuned LLM Application",
            "Text-to-Image Generation Tool",
            "Music Generation System",
            "Creative Writing AI Assistant",
            "Voice Synthesis Application"
        ],
        "agentic_ai": [
            "Autonomous Task Execution Agent",
            "Multi-Agent Simulation",
            "Tool-Using AI Assistant",
            "AI Agent for Data Analysis",
            "Agent-Based Decision Support System"
        ]
    },
    "Video Tutorials": {
        "python_beginner": [
            "Educational Python Basics Series",
            "Interactive Coding Tutorial Videos",
            "Python Concept Explanation Screencast",
            "Code-Along Project Videos",
            "Python Tips and Tricks Channel"
        ],
        "python_intermediate": [
            "Advanced Python Features Tutorial Series",
            "Framework Deep-Dive Videos",
            "Performance Optimization Screencasts",
            "Design Patterns in Python Series",
            "Testing and Debugging Tutorials"
        ],
        "data_science_beginner": [
            "Data Analysis Walkthrough Series",
            "Statistics Visualization Tutorials",
            "Data Cleaning Process Videos",
            "Basic Machine Learning Model Tutorials",
            "Data Visualization Guide Videos"
        ],
        "data_science_advanced": [
            "Advanced ML Algorithm Explanations",
            "Feature Engineering Masterclass",
            "Model Optimization Techniques Series",
            "ML Pipeline Development Videos",
            "Model Deployment Tutorials"
        ],
        "ai_specialization": [
            "Neural Network Architecture Explanations",
            "Deep Learning Framework Tutorials",
            "Computer Vision Project Series",
            "NLP Implementation Videos",
            "AI Ethics Discussion Series"
        ],
        "generative_ai": [
            "LLM Implementation Tutorials",
            "Diffusion Model Training Guide",
            "Prompt Engineering Masterclass",
            "Fine-tuning Walkthrough Videos",
            "Generative AI Application Tutorials"
        ],
        "agentic_ai": [
            "AI Agent Development Series",
            "Multi-Agent System Tutorials",
            "LangChain Implementation Videos",
            "Tool-Using AI Development Guide",
            "Agent Communication Protocol Tutorials"
        ]
    },
    "Interactive Exercises": {
        "python_beginner": [
            "Python Syntax Practice Platform",
            "Interactive Coding Challenge Website",
            "Function Implementation Exercises",
            "Python Puzzle Game",
            "Basic Algorithm Challenge Series"
        ],
        "python_intermediate": [
            "Object-Oriented Programming Exercises",
            "Advanced Data Structure Challenges",
            "Algorithm Optimization Problems",
            "Design Pattern Implementation Tasks",
            "Testing Framework Exercise Platform"
        ],
        "data_science_beginner": [
            "Data Cleaning Exercise Platform",
            "Statistical Analysis Practice Problems",
            "Basic ML Model Implementation Challenges",
            "Data Visualization Exercise Series",
            "Exploratory Data Analysis Worksheets"
        ],
        "data_science_advanced": [
            "Advanced ML Algorithm Implementation",
            "Feature Engineering Challenge Platform",
            "Model Optimization Exercises",
            "NLP Processing Tasks",
            "Time Series Analysis Problems"
        ],
        "ai_specialization": [
            "Neural Network Implementation Exercises",
            "Deep Learning Framework Challenges",
            "Computer Vision Task Series",
            "NLP Model Building Problems",
            "AI Ethics Case Studies"
        ],
        "generative_ai": [
            "Prompt Engineering Practice Platform",
            "LLM Fine-tuning Exercise Suite",
            "Diffusion Model Parameter Tuning",
            "Generative Model Evaluation Tasks",
            "Text-to-Image Generation Challenges"
        ],
        "agentic_ai": [
            "Agent Development Exercise Platform",
            "Multi-Agent System Building Tasks",
            "Tool-Using AI Implementation Challenges",
            "Reinforcement Learning Problems",
            "Agent Communication Protocol Exercises"
        ]
    },
    "Combination": {
        "python_beginner": [
            "Multi-format Python Learning Platform",
            "Integrated Code and Video Tutorial Project",
            "Interactive Documentation System",
            "Visual and Written Tutorial Combination",
            "Exercise-Based Learning Environment"
        ],
        "python_intermediate": [
            "Full-Stack Python Development Course",
            "Project-Based Learning Platform",
            "Video and Interactive Exercise Combination",
            "Visual Programming Environment",
            "Code Review and Mentoring System"
        ],
        "data_science_beginner": [
            "Data Science Learning Path Platform",
            "Interactive Data Analysis Environment",
            "Video and Exercise-Based Statistics Course",
            "Visual Data Science Notebook System",
            "Hands-on Data Project Platform"
        ],
        "data_science_advanced": [
            "Advanced ML Project Portfolio",
            "Interactive Research Implementation Platform",
            "Video and Code-Based ML Framework",
            "Visual ML Pipeline Development System",
            "Experimental ML Environment"
        ],
        "ai_specialization": [
            "AI Research and Implementation Platform",
            "Deep Learning Visual Learning System",
            "Interactive Neural Network Builder",
            "Video and Code-Based AI Framework",
            "Hands-on AI Ethics Learning Environment"
        ],
        "generative_ai": [
            "Generative AI Development Platform",
            "Interactive LLM Training Environment",
            "Visual Prompt Engineering System",
            "Model Fine-tuning Learning Path",
            "Creative AI Implementation Framework"
        ],
        "agentic_ai": [
            "Agent Development Environment",
            "Multi-Agent Simulation Platform",
            "Interactive Tool-Using AI Builder",
            "Video and Code-Based Agent Framework",
            "Experimental Agent Testing System"
        ]
    }
}

In [8]:
# User session data store
SESSION_DATA = {}

def save_session(session_id, data):
    """Save session data to SESSION_DATA global dictionary"""
    if session_id in SESSION_DATA:
        SESSION_DATA[session_id].update(data)
    else:
        SESSION_DATA[session_id] = data

    # Add timestamp for session tracking
    SESSION_DATA[session_id]["last_activity"] = datetime.now().isoformat()

def load_session(session_id):
    """Load session data from SESSION_DATA global dictionary"""
    return SESSION_DATA.get(session_id, {})

import openai

openai.api_key = ""
MODEL_NAME = "gpt-4"

SYSTEM_PROMPT = "You are an intelligent and helpful educational assistant."


def recommend_learning_path(age, goals, knowledge_level, interests):
    paths = []
    if "beginner" in knowledge_level.lower():
        if any(topic in interests.lower() for topic in ["python", "programming", "coding"]):
            paths.append("python_beginner")
        if any(topic in interests.lower() for topic in ["data", "analysis", "statistics"]):
            paths.append("data_science_beginner")
    elif "intermediate" in knowledge_level.lower():
        if any(topic in interests.lower() for topic in ["python", "programming", "coding"]):
            paths.append("python_intermediate")
        if any(topic in interests.lower() for topic in ["data", "analysis", "statistics"]):
            paths.append("data_science_advanced")
        if any(topic in interests.lower() for topic in ["ai", "machine learning", "deep learning"]):
            paths.append("ai_specialization")
        if any(topic in interests.lower() for topic in ["generative", "gpt", "llm", "diffusion"]):
            paths.append("generative_ai")
    elif "advanced" in knowledge_level.lower() or "expert" in knowledge_level.lower():
        if any(topic in interests.lower() for topic in ["ai", "machine learning", "deep learning"]):
            paths.append("ai_specialization")
        if any(topic in interests.lower() for topic in ["generative", "gpt", "llm", "diffusion"]):
            paths.append("generative_ai")
        if any(topic in interests.lower() for topic in ["agent", "autonomous", "planning"]):
            paths.append("agentic_ai")
    if any(topic in interests.lower() for topic in ["generative", "gpt", "llm", "diffusion"]):
        if "generative_ai" not in paths:
            paths.append("generative_ai")
    if any(topic in interests.lower() for topic in ["agent", "autonomous", "planning"]):
        if "agentic_ai" not in paths:
            paths.append("agentic_ai")
    if not paths:
        paths = ["python_beginner"]
    return [LEARNING_PATHS[path] for path in paths if path in LEARNING_PATHS]

def generate_quiz(topic, difficulty):
    quiz_prompt = f"""
    Generate a {difficulty} level quiz on {topic} with 3 multiple-choice questions.
    For each question, provide 4 options and indicate the correct answer.
    Format the quiz nicely with clear question numbering and option lettering.
    """

    response = openai.ChatCompletion.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": quiz_prompt}
        ]
    )

    return response["choices"][0]["message"]["content"]

def create_study_plan(topic, time_available, goals, knowledge_level):
    plan_prompt = f"""
    Create a structured study plan for learning {topic} with {time_available} hours per week available for study.
    The learner's goal is: {goals}
    The learner's knowledge level is: {knowledge_level}

    Include:
    1. Weekly breakdown of topics
    2. Time allocation for theory vs practice
    3. Recommended resources for each week
    4. Milestone projects or assessments
    5. Tips for effective learning

    Make this plan specific, actionable, and tailored to the knowledge level.
    """

    response = openai.ChatCompletion.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": plan_prompt}
        ]
    )

    return response["choices"][0]["message"]["content"]

def chat_with_model(user_input, session_id):
    user_data = load_session(session_id)

    context = ""
    if user_data:
        context = f"""
        User Profile:
        - Age: {user_data.get('age', 'Unknown')}
        - Knowledge Level: {user_data.get('knowledge_level', 'Unknown')}
        - Learning Goals: {user_data.get('goals', 'Unknown')}
        - Interests: {user_data.get('interests', 'Unknown')}
        - Available Study Time: {user_data.get('study_time', 'Unknown')} hours per week
        - Preferred Learning Style: {user_data.get('learning_style', 'Unknown')}

        Based on this profile, tailor your response appropriately.
        """

    chat_history = user_data.get('chat_history', [])
    if chat_history:
        context += "\n\nRecent conversation context (most recent first):\n"
        for i, (q, a) in enumerate(reversed(chat_history[-3:])):
            context += f"User: {q}\nYou: {a}\n\n"

    full_prompt = f"{context}\n\nUser's current question: {user_input}"

    response = openai.ChatCompletion.create(
        model=MODEL_NAME,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": full_prompt}
        ]
    )

    reply = response["choices"][0]["message"]["content"]

    if 'chat_history' not in user_data:
        user_data['chat_history'] = []
    user_data['chat_history'].append((user_input, reply))
    save_session(session_id, user_data)

    return reply

def chatbot_interface(session_id, message):
    if not message:
        return "Please type a message to start the conversation."

    return chat_with_model(message, session_id)

def handle_quiz_request(session_id, topic, difficulty):
    if not topic:
        return "Please specify a topic for the quiz."

    return generate_quiz(topic, difficulty)

def handle_study_plan_request(session_id, topic, time_available):
    if not topic:
        return "Please specify a topic for your study plan."

    profile = load_session(session_id)
    goals = profile.get("goals", "")
    knowledge_level = profile.get("knowledge_level", "Beginner")

    return create_study_plan(topic, time_available, goals, knowledge_level)


In [11]:
def user_onboarding(session_id, age, goals, knowledge_level, interests, study_time, learning_style):
    """Handle user profile submission and generate initial recommendations"""
    profile_data = {
        "age": age,
        "goals": goals,
        "knowledge_level": knowledge_level,
        "interests": interests,
        "study_time": study_time,
        "learning_style": learning_style
    }
    save_session(session_id, profile_data)

    # Generate initial recommendations based on the profile
    recommended_paths = recommend_learning_path(age, goals, knowledge_level, interests)
    recommendations = "### Recommended Learning Paths:\n\n"
    if recommended_paths:
        for path in recommended_paths:
            recommendations += f"- **{path['title']}**: {path['description']}\n"
            if path['modules']:
                recommendations += "  Modules: " + ", ".join(path['modules']) + "\n"
        recommendations += "\n\n"
    else:
        recommendations += "No specific learning paths recommended based on your profile. You can still explore the resources.\n\n"

    # Also generate resource recommendations based on learning style and paths
    recommendations += "### Recommended Resources:\n\n"
    if learning_style in LEARNING_RESOURCES:
        style_resources = LEARNING_RESOURCES[learning_style]
        for path_key in [p['title'] for p in recommended_paths]:
             # Find the key in LEARNING_PATHS that matches the path_key title
            matched_key = next((key for key, value in LEARNING_PATHS.items() if value['title'] == path_key), None)
            if matched_key and matched_key in style_resources:
                recommendations += f"#### For {path_key} ({learning_style} Style):\n"
                for resource in style_resources[matched_key]:
                    recommendations += f"- [{resource['title']}]({resource['url']})\n"
                recommendations += "\n"
    else:
         recommendations += f"No specific resources found for your preferred style: {learning_style}.\n\n"


    # Add project ideas
    recommendations += "### Project Ideas:\n\n"
    if learning_style in PROJECT_IDEAS:
         style_projects = PROJECT_IDEAS[learning_style]
         for path_key in [p['title'] for p in recommended_paths]:
             # Find the key in LEARNING_PATHS that matches the path_key title
            matched_key = next((key for key, value in LEARNING_PATHS.items() if value['title'] == path_key), None)
            if matched_key and matched_key in style_projects:
                recommendations += f"#### For {path_key} ({learning_style} Style):\n"
                for project in style_projects[matched_key]:
                    recommendations += f"- {project}\n"
                recommendations += "\n"
    else:
         recommendations += f"No specific project ideas found for your preferred style: {learning_style}.\n\n"


    return recommendations

def generate_recommendations(session_id):
    """Generate recommendations based on saved session data"""
    profile = load_session(session_id)
    if not profile:
        return "Please complete your profile first in the 'Profile & Goals' tab."

    age = profile.get("age", "")
    goals = profile.get("goals", "")
    knowledge_level = profile.get("knowledge_level", "Beginner")
    interests = profile.get("interests", "")
    learning_style = profile.get("learning_style", "Combination")


    # Generate initial recommendations based on the profile
    recommended_paths = recommend_learning_path(age, goals, knowledge_level, interests)
    recommendations = "### Recommended Learning Paths:\n\n"
    if recommended_paths:
        for path in recommended_paths:
            recommendations += f"- **{path['title']}**: {path['description']}\n"
            if path['modules']:
                recommendations += "  Modules: " + ", ".join(path['modules']) + "\n"
        recommendations += "\n\n"
    else:
        recommendations += "No specific learning paths recommended based on your profile. You can still explore the resources.\n\n"

    # Also generate resource recommendations based on learning style and paths
    recommendations += "### Recommended Resources:\n\n"
    if learning_style in LEARNING_RESOURCES:
        style_resources = LEARNING_RESOURCES[learning_style]
        for path_key in [p['title'] for p in recommended_paths]:
             # Find the key in LEARNING_PATHS that matches the path_key title
            matched_key = next((key for key, value in LEARNING_PATHS.items() if value['title'] == path_key), None)
            if matched_key and matched_key in style_resources:
                recommendations += f"#### For {path_key} ({learning_style} Style):\n"
                for resource in style_resources[matched_key]:
                    recommendations += f"- [{resource['title']}]({resource['url']})\n"
                recommendations += "\n"
    else:
         recommendations += f"No specific resources found for your preferred style: {learning_style}.\n\n"


    # Add project ideas
    recommendations += "### Project Ideas:\n\n"
    if learning_style in PROJECT_IDEAS:
         style_projects = PROJECT_IDEAS[learning_style]
         for path_key in [p['title'] for p in recommended_paths]:
             # Find the key in LEARNING_PATHS that matches the path_key title
            matched_key = next((key for key, value in LEARNING_PATHS.items() if value['title'] == path_key), None)
            if matched_key and matched_key in style_projects:
                recommendations += f"#### For {path_key} ({learning_style} Style):\n"
                for project in style_projects[matched_key]:
                    recommendations += f"- {project}\n"
                recommendations += "\n"
    else:
         recommendations += f"No specific project ideas found for your preferred style: {learning_style}.\n\n"


    return recommendations


def chatbot_interface(session_id, message):
    """Process user messages and generate assistant responses using a local model"""

    if not message:
        return "Please type a message to start the conversation."

    # Build system message with context from user profile if available
    system_message = SYSTEM_PROMPT

    user_data = load_session(session_id)

    if user_data:
        system_message += f"\n\nUser Profile Information:\n"
        system_message += f"- Knowledge Level: {user_data.get('knowledge_level', 'Unknown')}\n"
        system_message += f"- Learning Style: {user_data.get('learning_style', 'Unknown')}\n"
        system_message += f"- Interests: {user_data.get('interests', 'Unknown')}\n"
        system_message += f"- Learning Goals: {user_data.get('goals', 'Unknown')}\n"


    # Prepare prompt for generation
    # Use Groq client for chat completion
    try:
        chat_history = user_data.get('chat_history', [])
        messages = [{"role": "system", "content": system_message}]
        for q, a in chat_history[-5:]: # Include last 5 turns
            messages.append({"role": "user", "content": q})
            messages.append({"role": "assistant", "content": a})
        messages.append({"role": "user", "content": message})


        response = client.chat.completions.create(
            model="llama3-8b-8192",  # Or another suitable Groq model
            messages=messages,
            max_tokens=1024,
            temperature=0.7,
        )

        reply = response.choices[0].message.content

         # Save chat history
        if 'chat_history' not in user_data:
            user_data['chat_history'] = []
        user_data['chat_history'].append((message, reply))
        save_session(session_id, user_data)


        return reply

    except Exception as e:
        return f"Error during response generation: {str(e)}"


def handle_quiz_request(session_id, topic, difficulty):
    """Generate a quiz based on the specified topic and difficulty"""

    if not topic:
        return "Please specify a topic for the quiz."

    user_data = load_session(session_id)
    system_message = SYSTEM_PROMPT

    if user_data:
        system_message += f"\n\nUser Profile Information:\n"
        system_message += f"- Knowledge Level: {user_data.get('knowledge_level', 'Unknown')}\n"
        system_message += f"- Learning Style: {user_data.get('learning_style', 'Unknown')}\n"
        system_message += f"- Interests: {user_data.get('interests', 'Unknown')}\n"
        system_message += f"- Learning Goals: {user_data.get('goals', 'Unknown')}\n"

    # Build prompt for quiz generation
    prompt = f"Create a quiz on {topic} at {difficulty} level. Include 5 questions with multiple choice answers and provide the correct answers at the end."


    # Use Groq client for chat completion
    try:
        messages = [{"role": "system", "content": system_message}]
        messages.append({"role": "user", "content": prompt})

        response = client.chat.completions.create(
            model="llama3-8b-8192",  # Or another suitable Groq model
            messages=messages,
            max_tokens=512,
            temperature=0.7,
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"Error during quiz generation: {str(e)}"


def handle_study_plan_request(session_id, topic, time_available):
    """Generate a personalized study plan based on topic and available time"""

    if not topic:
        return "Please specify a topic for your study plan."

    user_data = load_session(session_id)
    system_message = SYSTEM_PROMPT

    if user_data:
        system_message += f"\n\nUser Profile Information:\n"
        system_message += f"- Knowledge Level: {user_data.get('knowledge_level', 'Unknown')}\n"
        system_message += f"- Learning Style: {user_data.get('learning_style', 'Unknown')}\n"
        system_message += f"- Interests: {user_data.get('interests', 'Unknown')}\n"
        system_message += f"- Learning Goals: {user_data.get('goals', 'Unknown')}\n"


    # Build prompt for study plan generation
    prompt = f"Create a detailed study plan for learning {topic} with {time_available} hours available per week. Include specific goals, resources, and milestones."


    # Use Groq client for chat completion
    try:
        messages = [{"role": "system", "content": system_message}]
        messages.append({"role": "user", "content": prompt})

        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=messages,
            max_tokens=1024,
            temperature=0.7,
        )

        return response.choices[0].message.content

    except Exception as e:
        return f"Error during study plan generation: {str(e)}"


def add_generative_ai_info():
    """Return information about Generative AI"""
    return """
    ## What is Generative AI?

    Generative AI refers to artificial intelligence systems that can create new content, such as text, images, code, audio, video, or 3D models. Unlike traditional AI systems that are designed to recognize patterns or make predictions, generative AI creates original outputs based on the patterns it has learned during training.

    ### Key Concepts in Generative AI:

    - *Large Language Models (LLMs)*: Text generation systems like GPT-4, LLaMA, Claude, etc.
    - *Diffusion Models*: For image generation (DALL-E, Midjourney, Stable Diffusion)
    - *Prompt Engineering*: The art of crafting inputs to get desired outputs
    - *Fine-tuning*: Adapting pre-trained models for specific domains or tasks
    - *RLHF (Reinforcement Learning from Human Feedback)*: Method for aligning AI with human preferences

    Learning generative AI involves understanding these foundation models, how they work, and how to effectively use and customize them for various applications.
    """

def add_agentic_ai_info():
    """Return information about Agentic AI"""
    return """
    ## What is Agentic AI?

    Agentic AI refers to AI systems that can act autonomously to achieve specified goals. Unlike passive AI systems that respond only when prompted, agentic AI can take initiative, make decisions, use tools, and perform sequences of actions to accomplish tasks.

    ### Key Concepts in Agentic AI:

    - *Planning & Decision Making*: AI systems that can formulate and execute plans
    - *Tool Use*: AI that can leverage external tools and APIs
    - *Autonomous Execution*: Systems that can work without constant human supervision
    - *Multi-agent Systems*: Multiple AI agents collaborating or competing
    - *Memory & Context Management*: How agents maintain state across interactions

    Agentic AI represents an evolution from AI as a passive tool to AI as an active collaborator that can work independently while remaining aligned with human goals and values.
    """


def create_chatbot():
    """Create the Gradio interface for the chatbot"""
    # Generate a random session ID for the user
    session_id = str(uuid.uuid4())

    # Define theme colors
    theme_colors = {
        "light": {
            "primary": "#4a6fa5",
            "secondary": "#6c757d",
            "success": "#28a745",
            "background": "#f8f9fa",
            "text": "#212529",
            "card_bg": "#ffffff",
            "card_border": "#dee2e6",
            "input_bg": "#ffffff",
            "highlight": "#e7f5fe",
            "accent": "#007bff",
            "completed": "#d4edda",
            "info_box_bg": "#e7f5fe"
        },
        "dark": {
            "primary": "#5b88c7",         # Lighter blue for better visibility in dark mode
            "secondary": "#adb5bd",       # Lighter gray for better visibility
            "success": "#48c774",         # Brighter green for dark mode
            "background": "#1a1a1a",      # Dark background
            "text": "#f1f1f1",            # Light text for dark backgrounds
            "card_bg": "#2d2d2d",         # Dark card background
            "card_border": "#444444",     # Dark border
            "input_bg": "#333333",        # Dark input background
            "highlight": "#193652",       # Dark highlight
            "accent": "#3291ff",          # Bright accent
            "completed": "#204829",       # Dark green for completed
            "info_box_bg": "#193652"      # Dark info box
        }
    }

    # Create CSS with theme variables
    custom_css = """
        /* Theme variables - will be applied based on user's theme preference */
        .light-theme {
            --primary-color: #4a6fa5;
            --secondary-color: #6c757d;
            --success-color: #28a745;
            --bg-color: #f8f9fa;
            --text-color: #212529;
            --card-bg: #ffffff;
            --card-border: #dee2e6;
            --input-bg: #ffffff;
            --highlight-color: #e7f5fe;
            --accent-color: #007bff;
            --completed-color: #d4edda;
            --info-box-bg: #e7f5fe;
        }

        .dark-theme {
            --primary-color: #5b88c7;
            --secondary-color: #adb5bd;
            --success-color: #48c774;
            --bg-color: #1a1a1a;
            --text-color: #f1f1f1;
            --card-bg: #2d2d2d;
            --card-border: #444444;
            --input-bg: #333333;
            --highlight-color: #193652;
            --accent-color: #3291ff;
            --completed-color: #204829;
            --info-box-bg: #193652;
        }

        /* Automatically detect theme preference and apply appropriate theme class */
        @media (prefers-color-scheme: dark) {
            body {
                color-scheme: dark;
            }

            body:not(.light-theme):not(.force-light) {
                background-color: var(--bg-color);
                color: var(--text-color);
            }
        }

        @media (prefers-color-scheme: light) {
            body {
                color-scheme: light;
            }

            body:not(.dark-theme):not(.force-dark) {
                background-color: var(--bg-color);
                color: var(--text-color);
            }
        }

        /* Apply theme variables to elements */
        .gradio-container {
            background-color: var(--bg-color) !important;
            color: var(--text-color) !important;
            font-family: 'Inter', 'Segoe UI', sans-serif;
        }

        /* Style overrides for Gradio components to respect theme */
        .gr-box, .gr-panel {
            background-color: var(--card-bg) !important;
            border-color: var(--card-border) !important;
            color: var(--text-color) !important;
        }

        .gr-input, .gr-dropdown {
            background-color: var(--input-bg) !important;
            color: var(--text-color) !important;
            border-color: var(--card-border) !important;
        }

        .gr-input:focus, .gr-dropdown:focus {
            border-color: var(--accent-color) !important;
        }

        .gr-input::placeholder {
            color: var(--secondary-color) !important;
            opacity: 0.7;
        }

        /* App specific styling */
        #title {
            font-size: 32px;
            font-weight: bold;
            text-align: center;
            padding-top: 20px;
            color: var(--primary-color) !important;
            margin-bottom: 0;
        }

        #subtitle {
            font-size: 18px;
            text-align: center;
            margin-bottom: 20px;
            color: var(--secondary-color) !important;
        }

        .card {
            background-color: var(--card-bg) !important;
            color: var(--text-color) !important;
            padding: 20px;
            border-radius: 12px;
            border: 1px solid var(--card-border);
            box-shadow: 0 4px 10px rgba(0,0,0,0.08);
            margin-bottom: 20px;
        }

        /* Fix markdown text color for both themes */
        .prose {
            color: var(--text-color) !important;
        }

        /* Ensure markdown headings are visible in both themes */
        .prose h1, .prose h2, .prose h3, .prose h4, .prose h5, .prose h6 {
            color: var(--primary-color) !important;
            font-weight: 600;
        }

        /* Make links visible in both themes */
        .prose a {
            color: var(--accent-color) !important;
            text-decoration: underline;
        }

        /* Fix button colors */
        .gr-button-primary {
            background-color: var(--primary-color) !important;
            color: #ffffff !important;
        }

        .gr-button-secondary {
            background-color: var(--secondary-color) !important;
            color: #ffffff !important;
        }

        .gr-button-success {
            background-color: var(--success-color) !important;
            color: #ffffff !important;
        }

        /* Footer styling */
        .footer {
            text-align: center;
            margin-top: 30px;
            padding: 10px;
            font-size: 14px;
            color: var(--secondary-color) !important;
        }

        /* Progress modules */
        .progress-module {
            padding: 10px;
            margin: 5px 0;
            border-radius: 5px;
            background-color: var(--highlight-color);
            color: var(--text-color);
        }

        .progress-module.completed {
            background-color: var(--completed-color);
        }

        /* Info box styling */
        .info-box {
            background-color: var(--info-box-bg);
            border-left: 4px solid var(--primary-color);
            padding: 15px;
            margin: 15px 0;
            border-radius: 4px;
            color: var(--text-color);
        }

        /* Tab styling improvements */
        .tabs button {
            color: var(--text-color) !important;
        }

        .tabs button[data-selected="true"] {
            color: var(--primary-color) !important;
            border-color: var(--primary-color) !important;
        }

        /* Add theme detection script */
        .theme-script {
            display: none;
        }
    """

    # JavaScript to detect and apply theme
    theme_script = """
    <script>
        // Function to detect and apply theme
        function applyTheme() {
            // Check if user has a saved preference
            const savedTheme = localStorage.getItem('preferredTheme');

            if (savedTheme) {
                // Apply saved preference
                document.body.classList.add(savedTheme + '-theme');
            } else {
                // Check system preference
                if (window.matchMedia && window.matchMedia('(prefers-color-scheme: dark)').matches) {
                    document.body.classList.add('dark-theme');
                } else {
                    document.body.classList.add('light-theme');
                }
            }

            // Listen for theme changes
            window.matchMedia('(prefers-color-scheme: dark)').addEventListener('change', event => {
                // Only apply if no saved preference
                if (!savedTheme) {
                    document.body.classList.remove('light-theme', 'dark-theme');
                    document.body.classList.add(event.matches ? 'dark-theme' : 'light-theme');
                }
            });
        }

        // Apply theme when DOM is loaded
        document.addEventListener('DOMContentLoaded', applyTheme);

        // For Gradio that might load content dynamically
        if (document.readyState === 'complete' || document.readyState === 'interactive') {
            setTimeout(applyTheme, 1);
        }
    </script>
    """

    with gr.Blocks(css=custom_css, theme=gr.themes.Soft(primary_hue="blue")) as demo:
        gr.HTML("<div id='title'>🎓 AI Teaching Assistant</div>")
        gr.HTML("<div id='subtitle'>Your personalized learning companion for Python, Data Science & AI</div>")
        gr.HTML(theme_script, elem_classes=["theme-script"])

        # Tabs for different sections
        with gr.Tabs(elem_classes=["tabs"]) as tabs:
            # Profile Tab
            with gr.Tab("Profile & Goals"):
                with gr.Column(elem_classes=["card"]):
                    gr.HTML("<h3>Complete Your Learning Profile</h3>")

                    with gr.Row():
                        with gr.Column(scale=1):
                            age_input = gr.Textbox(
                                label="Age",
                                placeholder="e.g. 20",
                                lines=1
                            )
                        with gr.Column(scale=2):
                            knowledge_level_input = gr.Dropdown(
                                choices=["Beginner", "Intermediate", "Advanced", "Expert"],
                                label="Knowledge Level",
                                value="Beginner"
                            )

                    goals_input = gr.Textbox(
                        label="Learning Goals",
                        placeholder="e.g. I want to become a data scientist and work with machine learning models",
                        lines=2
                    )

                    interests_input = gr.Textbox(
                        label="Specific Interests",
                        placeholder="e.g. Python, data visualization, neural networks",
                        lines=2
                    )

                    with gr.Row():
                        with gr.Column(scale=1):
                            study_time_input = gr.Dropdown(
                                choices=["1-3", "4-6", "7-10", "10+"],
                                label="Hours Available Weekly",
                                value="4-6"
                            )
                        with gr.Column(scale=2):
                            learning_style_input = gr.Dropdown(
                                choices=["Visual", "Reading/Writing", "Hands-on Projects", "Video Tutorials", "Interactive Exercises", "Combination"],
                                label="Preferred Learning Style",
                                value="Combination"
                            )

                    profile_submit_btn = gr.Button("Save Profile & Generate Recommendations", variant="primary")
                    profile_output = gr.Markdown(label="Personalized Recommendations")

            # Chat Tab
            with gr.Tab("Learning Assistant"):
                with gr.Row():
                    with gr.Column(elem_classes=["card"]):
                        chat_input = gr.Textbox(
                            label="Ask a Question",
                            placeholder="Ask anything about Python, Data Science, AI...",
                            lines=3
                        )

                        with gr.Row():
                            chat_submit_btn = gr.Button("Send Message", variant="primary")
                            chat_clear_btn = gr.Button("Clear Chat", variant="secondary")

                        chat_output = gr.Markdown(label="Assistant Response")

            # Resources Tab
            with gr.Tab("Resources & Recommendations"):
                with gr.Column(elem_classes=["card"]):
                    gr.HTML("<h3>Your Learning Resources</h3>")
                    refresh_recommendations_btn = gr.Button("Refresh Recommendations", variant="primary")
                    recommendations_output = gr.Markdown(label="Personalized Recommendations")

            # Practice Tab
            with gr.Tab("Practice & Assessment"):
                with gr.Column(elem_classes=["card"]):
                    gr.HTML("<h3>Generate a Quiz</h3>")

                    with gr.Row():
                        quiz_topic_input = gr.Textbox(
                            label="Quiz Topic",
                            placeholder="e.g. Python Lists",
                            lines=1
                        )
                        quiz_difficulty_input = gr.Dropdown(
                            choices=["Beginner", "Intermediate", "Advanced"],
                            label="Difficulty Level",
                            value="Beginner"
                        )

                    generate_quiz_btn = gr.Button("Generate Quiz", variant="primary")
                    quiz_output = gr.Markdown(label="Quiz")

            # Study Plan Tab
            with gr.Tab("Study Plan"):
                with gr.Column(elem_classes=["card"]):
                    gr.HTML("<h3>Generate a Personalized Study Plan</h3>")

                    with gr.Row():
                        plan_topic_input = gr.Textbox(
                            label="Study Topic",
                            placeholder="e.g. Data Science",
                            lines=1
                        )
                        plan_time_input = gr.Dropdown(
                            choices=["1-3", "4-6", "7-10", "10+"],
                            label="Hours Available Weekly",
                            value="4-6"
                        )

                    generate_plan_btn = gr.Button("Generate Study Plan", variant="primary")
                    plan_output = gr.Markdown(label="Personalized Study Plan")

        gr.HTML("""<div class="footer">
            AI Teaching Assistant |Created by Aether AI Team
        </div>""")

        # Event handlers
        profile_submit_btn.click(
            user_onboarding,
            inputs=[
                gr.State(session_id),
                age_input,
                goals_input,
                knowledge_level_input,
                interests_input,
                study_time_input,
                learning_style_input
            ],
            outputs=profile_output
        )

        chat_submit_btn.click(
            chatbot_interface,
            inputs=[gr.State(session_id), chat_input],
            outputs=chat_output
        )

        chat_clear_btn.click(
            lambda: "",
            inputs=[],
            outputs=[chat_output, chat_input]
        )

        refresh_recommendations_btn.click(
            generate_recommendations,
            inputs=[gr.State(session_id)],
            outputs=recommendations_output
        )

        generate_plan_btn.click(
            handle_study_plan_request,
            inputs=[gr.State(session_id), plan_topic_input, plan_time_input],
            outputs=plan_output
        )

    return demo

if __name__ == "__main__":
    app = create_chatbot()
    app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0612e01dffd4af68c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
